<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
from torch.autograd import Variable

class Controller(nn.Module):
    def __init__(self, num_choices, num_layers):
        super(Controller, self).__init__()
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=num_choices, hidden_size=32)
        self.fc = nn.Linear(32, num_choices)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[-1])
        return torch.softmax(out, dim=-1)

class SimpleNetwork(nn.Module):
    def __init__(self, architecture):
        super(SimpleNetwork, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(architecture[i], architecture[i + 1]) for i in range(len(architecture) - 1)])

    def forward(self, x):
        for layer in self.layers:
            x = torch.relu(layer(x))
        return x

def evaluate_model(model):
    # Placeholder for model evaluation logic
    # Train the model on some data and return its performance
    # For illustration, we'll return a random performance value
    performance = random.uniform(0.1, 1.0)
    return torch.tensor(performance, requires_grad=True)

def train_controller(controller, architecture_fn, num_choices, num_layers, num_samples=10):
    optimizer = optim.Adam(controller.parameters(), lr=0.01)
    for _ in range(num_samples):
        architecture = [random.choice(range(num_choices)) for _ in range(num_layers)]
        model = architecture_fn(architecture)
        performance = evaluate_model(model)
        loss = -torch.log(performance)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Architecture: {architecture}, Performance: {performance.item()}')

# Example usage
num_choices = 5
num_layers = 3
controller = Controller(num_choices, num_layers)
train_controller(controller, lambda arch: SimpleNetwork([10] + arch + [1]), num_choices, num_layers)